### >>>>>>>>>> for using interactive features please fork it and then run <<<<<<<<<<

![](https://www.osicild.org/uploads/1/2/2/7/122798879/editor/kaggle-v01-clipped_2.png?1569348761)
## If you want to know details about the Pulmonary Fibrosis Progression, check out my other kernel [Pulmonary Fibrosis for Non-Med People](https://www.kaggle.com/redwankarimsony/pulmonary-fibrosis-for-non-med-people) which contains detailed information about the the disease, symptoms, causes and many more. If you like this kernel, please upvote.. 

<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">Table of content</h3>
<font color="teal" size=+1><b>Part 1: EDA</b></font><br>  
    
&#9632; [1. Load Dataframes](#1)<br>
&#9632; [2. Train and Test Dataframe Details](#2)<br>
&#9632; [3. Number of Patients](#3)<br>
&#9632; [4. Creating Individual Patient Profiles ](#4)<br>
&#9632; [5. Gender Distribution](#5)<br>
&#9632; [6. Age Distribution of the Patients](#6)<br>
&#9632; [7. Study of SmokingStatus](#7)<br>
&#9632; [8. Study of Gender vs SmokingStatus](#8)<br>
&#9632; [9. Age Distribution vs SmokingStatus](#9)<br>
&#9632; [10. Interactive Patient Lookup (FVC Decay)](#10)<br>
&#9632; [11. Interactive Patient Lookup (CT Scans)](#11)<br>
&#9632; [12. Sumarizing the Unique Patient Profile](#12)<br>

    
<font color="teal" size=+1><b>Part 2: Lung Segmentation</b></font><br>
&#9632; [13. Segmentation of Lung](#13)

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


#plotly imports
!pip install chart_studio
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

<a id="1"></a>
<font color="black" size=+2.5><b>1. Load Dataframes</b></font>

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Go to TOC</a>

In [ ]:
train_dir = '../input/osic-pulmonary-fibrosis-progression/train/'
test_dir = '../input/osic-pulmonary-fibrosis-progression/test/'


train_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
test_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')
print('Training Dataframe shape: ', train_df.shape)

train_df.head(10)

<a id="2"></a>
<font color="black" size=+2.5><b>2. Train and Test Dataframe Details</b></font>

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Go to TOC</a>

In [ ]:
# Let's have a look at the detailed info about the dataframes
print('Training Dataframe Details: ')
print(train_df.info())

print('\n\nTest Dataframe Details: ')
print(test_df.info())





<a id="3"></a>
<font color="black" size=+2.5><b>3. Number of Patients</b></font>

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Go to TOC</a>

In [ ]:
print('Number of patients in training set:',
      len(os.listdir(train_dir)))
print('Number of patients in test set:',
     len(os.listdir(test_dir)))

<a id="4"></a>
<font color="black" size=+2.5><b>4. Creating Individual Patient Profiles </b></font>

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Go to TOC</a>

Let's create a dataframe that will contain all the unique patient IDs. 

In [ ]:
# Creating unique patient lists and their properties. 
patient_ids = os.listdir(train_dir)
patient_ids = sorted(patient_ids)

#Creating new rows
no_of_instances = []
age = []
sex = []
smoking_status = []

for patient_id in patient_ids:
    patient_info = train_df[train_df['Patient'] == patient_id].reset_index()
    no_of_instances.append(len(os.listdir(train_dir + patient_id)))
    age.append(patient_info['Age'][0])
    sex.append(patient_info['Sex'][0])
    smoking_status.append(patient_info['SmokingStatus'][0])

#Creating the dataframe for the patient info    
patient_df = pd.DataFrame(list(zip(patient_ids, no_of_instances, age, sex, smoking_status)), 
                                 columns =['Patient', 'no_of_instances', 'Age', 'Sex', 'SmokingStatus'])
print(patient_df.info())
patient_df.head()

<a id="5"></a>
<font color="black" size=+2.5><b>5. Gender Distribution</b></font>

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Go to TOC</a>

It seems that among the unique patients 78% are Male and rest of them are female. 


In [ ]:
patient_df['Sex'].value_counts(normalize = True).iplot(kind = 'bar', 
                                                        color = 'blue', 
                                                        yTitle = 'Unique patient count',
                                                        xTitle = 'Gender',
                                                        title = 'Gender Distribution of the unique patients')

<a id="6"></a>
<font color="black" size=+2.5><b>6. Age Distribution of the Patients</b></font>

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Go to TOC</a>

Now let's have a look at the patients age distributions. We observe that the age distribution starts from near 50 years and it tops to 90 years. Therefore, the given data about the pulmonary fibrosis is mainly of the older people.  

In [ ]:
import scipy

data = patient_df.Age.tolist()
plt.figure(figsize=(18,6))
# Creating the main histogram
_, bins, _ = plt.hist(data, 15, density=1, alpha=0.5)

# Creating the best fitting line with mean and standard deviation
mu, sigma = scipy.stats.norm.fit(data)
best_fit_line = scipy.stats.norm.pdf(bins, mu, sigma)
plt.plot(bins, best_fit_line, color = 'b', linewidth = 3, label = 'fitting curve')
plt.title(f'Age Distribution [ mean = {"{:.2f}".format(mu)}, standard_dev = {"{:.2f}".format(sigma)} ]', fontsize = 18)
plt.xlabel('Age -->')
plt.show()

patient_df['Age'].iplot(kind='hist',bins=25,color='blue',xTitle='Percent distribution',yTitle='Count')

let's see the average age of patients based on their sex. However, it is observed that the average age of male and female patients are almost same. However, male patients are mostly sick near the age of 70 

In [ ]:
plt.figure(figsize=(16, 6))
sns.kdeplot(patient_df.loc[patient_df['Sex'] == 'Male', 'Age'], label = 'Male',shade=True)
sns.kdeplot(patient_df.loc[patient_df['Sex'] == 'Female', 'Age'], label = 'Female',shade=True)
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');

<a id="7"></a>
<font color="black" size=+2.5><b>7. Study of SmokingStatus </b></font>

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Go to TOC</a>

It seems that almost 70% of the patients are ex-smokers (at some point of their lives, they smoked). However, among the patients about 27% of the patients have never smoked in their life. Hilarious thing is that 5% of the patients are still smoking.. LOL

In [ ]:
patient_df['SmokingStatus'].value_counts(normalize=True).iplot(kind='bar',
                                                      yTitle='Percentage', 
                                                      linecolor='black', 
                                                      opacity=0.8,
                                                      color='blue',
                                                      theme='pearl',
                                                      bargap=0.5,
                                                      title='SmokingStatus Distribution')

<a id="8"></a>
<font color="black" size=+2.5><b>8. Study of Gender vs SmokingStatus</b></font>

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Go to TOC</a>

In [ ]:
patient_df.groupby(['SmokingStatus', 'Sex']).count()['Patient'].unstack().iplot(kind='bar', 
                                                                                yTitle = 'Unique Patient Count',
                                                                                title = 'Gender vs SmokingStatus' )


<a id="9"></a>
<font color="black" size=+2.5><b>9. Age Distribution vs Smoking Status</b></font>

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Go to TOC</a>

In [ ]:
plt.figure(figsize=(16, 6))
sns.kdeplot(patient_df.loc[patient_df['SmokingStatus'] == 'Ex-smoker', 'Age'], label = 'Ex-smoker',shade=True)
sns.kdeplot(patient_df.loc[patient_df['SmokingStatus'] == 'Never smoked', 'Age'], label = 'Never smoked',shade=True)
sns.kdeplot(patient_df.loc[patient_df['SmokingStatus'] == 'Currently smokes', 'Age'], label = 'Currently smokes',shade=True)
# Labeling of plot
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');

<a id="10"></a>
<font color="black" size=+2.5><b>10. Interactive Patient Lookup (FVC Decay)</b></font>

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Go to TOC</a>

If you run the following cell, you will get an dropdown menu where you can select any patient ID and it will show you the details present for that patient in the training data frame. It will show you all the information of that patient from the train data frame and also plot the gradual fibrosis progression over time. It plots the decrease of FVC over time.  

#### Observations:
&#9632;  ***Smoking Status doesn't change over time.*** So smoking status for a single patient is always unique.<br> 
&#9632; ***Age of the patient doesn't change over time*** Whatever the age in the beginning of the diagnosis, even after more than 52 weeks patient's age didn't change in the dataframe. 


In [ ]:
from ipywidgets import interact  #, interactive, IntSlider, ToggleButtons

def patient_lookup(patient_id):
    print(train_df[train_df['Patient'] == patient_id])
    patient_info = train_df[train_df['Patient'] == patient_id].reset_index()
    fig, (ax1,ax2) = plt.subplots(1,2, figsize = (15, 5))
    ax1.plot(patient_info['Weeks'].tolist() , patient_info['FVC'].tolist(), marker = '*', linewidth = 3,color = 'r', markeredgecolor = 'b')
    ax1.set_title('FVC Deterioriation over the Weeks')
    ax1.set_xlabel('Weeks -->')
    ax1.set_ylabel('FVC')
    ax1.grid(True)
    
    ax2.plot(patient_info['Weeks'].tolist() , patient_info['Percent'].tolist(),marker = '*', linewidth = 3,
            color = 'r', markeredgecolor = 'b' )
    ax2.set_title('Percent change over the weeks')
    ax2.set_xlabel('Weeks -->')
    ax2.set_ylabel('Percent(of adult capacity)')
    ax2.grid(True)
    fig.suptitle(f'P_ID: {patient_id}', fontsize = 20) 
    
    
    
interact(patient_lookup, patient_id = patient_ids)

<a id="11"></a>
<font color="black" size=+2.5><b>11. Interactive Patient Lookup (CT Scans)</b></font>

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Go to TOC</a><br>
Here from the dropdown menu, please select any patient id, and it will show you 16 CT-Scans over time for that particular patient. <font color="red">Remember it's an interactive notebook. In order to use the interactive feature, you have to fork it and run it in your own instance in interactive mode.</font>

In [ ]:
import random
import pydicom
def explore_dicoms(patient_id, instance):
    RefDs = pydicom.dcmread('../input/osic-pulmonary-fibrosis-progression/train/' + 
                            patient_id +
                            '/' + 
                            str(instance) + '.dcm')
    plt.figure(figsize=(10, 5))
    plt.imshow(RefDs.pixel_array, cmap='gray');
    plt.title(f'P_ID: {patient_id}\nInstance: {instance}')
    plt.axis('off')


def show_ct_scans(patient_id):
    no_of_instances = int(patient_df[patient_df['Patient'] == patient_id]['no_of_instances'].values[0])
    files = sorted(random.sample(range(1, no_of_instances), 9))
    rows = 3
    cols = 3
    fig = plt.figure(figsize=(12,12))
    for idx in range(1, rows*cols+1):
        fig.add_subplot(rows, cols, idx)
        RefDs = pydicom.dcmread(train_dir + patient_id + '/' + str(files[idx-1]) + '.dcm')
        plt.imshow(RefDs.pixel_array, cmap='gray')
        plt.title(f'Instance: {files[idx-1]}')
        plt.axis(False)
        fig.add_subplot
    fig.suptitle(f'P_ID: {patient_id}') 
    plt.show()

In [ ]:
# show_ct_scans(patient_ids[0])
interact(show_ct_scans,patient_id = patient_ids)

<a id="12"></a>
<font color="black" size=+2.5><b>12. Sumarizing the Unique Patient Profile</b></font>

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Go to TOC</a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas_profiling as pdp
unique_patient_profile  = pdp.ProfileReport(patient_df)

# Part 2:

<a id="13"></a>
* <font color="black" size=+2.5><b>13. Lung Segmentation</b></font>

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Go to TOC</a>

For the lung segmentation, let's select any patient randomly and then we will see the CT-Scans of that patient. 

In [ ]:
patient_ids = os.listdir('../input/osic-pulmonary-fibrosis-progression/train/')
patient_id = 'ID00267637202270790561585'
dicom_filenames = os.listdir('../input/osic-pulmonary-fibrosis-progression/train/' + patient_id)
dicom_paths = ['../input/osic-pulmonary-fibrosis-progression/train/' + patient_id + '/'+ file for file in dicom_filenames]
    
def load_scan(paths):
    slices = [pydicom.read_file(path ) for path in paths]
    slices.sort(key = lambda x: int(x.InstanceNumber), reverse = True)
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness 
    return slices

def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    image = image.astype(np.int16)
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)   
    image += np.int16(intercept)
    return np.array(image, dtype=np.int16)

In [ ]:
# set path and load files 
patient_dicom = load_scan(dicom_paths)
patient_pixels = get_pixels_hu(patient_dicom)
#sanity check
plt.imshow(patient_pixels[46], cmap=plt.cm.bone)
plt.axis(False)
plt.show()

Lets use some thresholding and morphological operations to segment just the lung from the chest:

In [ ]:
# skimage image processing packages
from skimage import measure, morphology
from skimage.morphology import ball, binary_closing
from skimage.measure import label, regionprops
import copy


def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)
    counts = counts[vals != bg]
    vals = vals[vals != bg]
    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None
    
def segment_lung_mask(image, fill_lung_structures=True):
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = np.array(image >= -700, dtype=np.int8)+1
    labels = measure.label(binary_image)
 
    # Pick the pixel in the very corner to determine which label is air.
    # Improvement: Pick multiple background labels from around the  patient
    # More resistant to “trays” on which the patient lays cutting the air around the person in half
    background_label = labels[0,0,0]
 
    # Fill the air around the person
    binary_image[background_label == labels] = 2
 
    # Method of filling the lung structures (that is superior to 
    # something like morphological closing)
    if fill_lung_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
 
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1
 
    # Remove other air pockets inside body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0
 
    return binary_image

By running the code below, you are using skimage functions from above to create a mask that covers the lung. We will use both `fill_lung_structures=True` and `fill_lung_structures=False`, to isolate the lung and the internal structures. Let’s run the code below, and display an example of isolating the lung from the chest:

In [ ]:
# get masks 
segmented_lungs = segment_lung_mask(patient_pixels, fill_lung_structures=False)
segmented_lungs_fill = segment_lung_mask(patient_pixels, fill_lung_structures=True)
internal_structures = segmented_lungs_fill - segmented_lungs

# isolate lung from chest
copied_pixels = copy.deepcopy(patient_pixels)
for i, mask in enumerate(segmented_lungs_fill): 
    get_high_vals = mask == 0
    copied_pixels[i][get_high_vals] = 0
seg_lung_pixels = copied_pixels
# sanity check
f, ax = plt.subplots(1,2, figsize=(10,6))
ax[0].imshow(patient_pixels[46], cmap=plt.cm.bone)
ax[0].axis(False)
ax[0].set_title('Original')
ax[1].imshow(seg_lung_pixels[46], cmap=plt.cm.bone)
ax[1].axis(False)
ax[1].set_title('Segmented')
plt.show()

### 2D Visualizations Techniques
**Non Interactive:**
When visualizing data, I find very beneficial to visualize each process of your script. This will not only help you understand each step of your code, but it makes for a very nice, clean presentation.
The code below will, in essence, visually convey a story on how you segmented your data: 

&#9632; original image <br>
&#9632; the binary mask that covers the lung <br>
&#9632; highlighting internal structures of the lung using one of the masks <br>

In [ ]:
f, ax = plt.subplots(2,2, figsize = (10,10))

# pick random slice 
slice_id = 46

ax[0,0].imshow(patient_pixels[slice_id], cmap=plt.cm.bone)
ax[0,0].set_title('Original Dicom')
ax[0,0].axis(False)


ax[0,1].imshow(segmented_lungs_fill[slice_id], cmap=plt.cm.bone)
ax[0,1].set_title('Lung Mask')
ax[0,1].axis(False)

ax[1,0].imshow(seg_lung_pixels[slice_id], cmap=plt.cm.bone)
ax[1,0].set_title('Segmented Lung')
ax[1,0].axis(False)

ax[1,1].imshow(seg_lung_pixels[slice_id], cmap=plt.cm.bone)
ax[1,1].imshow(internal_structures[slice_id], cmap='jet', alpha=0.7)
ax[1,1].set_title('Segmentation with \nInternal Structure')
ax[1,1].axis(False)

### Interactive(1):
There are really only a few ways to display multiple images on jupyter notebook — manually plot each image one by one, or make a plot with multiple columns and rows to display in one figure. Unfortunately, this is not helpful for those that want to scan through each image and get a better understanding of the data. With the code below you create an interactive slide bar that lets you scroll through the images <font color=red>(But the catch is that you have to run the code in **interactive window**. So if you want to use the slider window to browse through the slices, fork the code and run it manually in interactive mode)</font>:

In [ ]:
# slide through dicom images using a slide bar 
plt.figure(1)
def dicom_animation(x):
    plt.imshow(patient_pixels[x], cmap = plt.cm.gray)
    return x
interact(dicom_animation, x=(0, len(patient_pixels)-1))

## Interactive(2):
Another way to visualize the CT-Angiograms in a bit lively fashion is to use gif images. Basically GIFs are a series of images shown at an preconfigured interval automatically. <font color=blue>However, the upper side of GIF is that, it stores images in lossless compression. </blue>. So it is highly efficent and accurate for storing CT scan slides.

In [ ]:
import imageio
from IPython import display
print('Original Image Slices before processing')
imageio.mimsave(f'./{patient_id}.gif', patient_pixels, duration=0.1)
display.Image(f'./{patient_id}.gif', format='png')

In [ ]:
print('Lung Segmentation Mask')
imageio.mimsave(f'./{patient_id}.gif', segmented_lungs_fill, duration=0.1)
display.Image(f'./{patient_id}.gif', format='png')

In [ ]:
print('Segmented Part of Lung Tissue')
imageio.mimsave(f'./{patient_id}.gif', seg_lung_pixels, duration=0.1)
display.Image(f'./{patient_id}.gif', format='png')

However, among the previous 3 GIFs, one of the most important is the Lung Segmentation Mask. You can see there are several images where we can see that the mask only selects the tissue portion of the lung but it is not considering the air vessels thorugh the lungs i.e. Bronchioles.

<br>
<font color=blue>(**Bronchioles** are air passages inside the lungs that branch off like tree limbs from the bronchi—the two main air passages into which air flows from the trachea (windpipe) after being inhaled through the nose or mouth. The bronchioles deliver air to tiny sacs called alveoli where oxygen and carbon dioxide are exchanged. They are vulnerable to conditions like asthma, bronchiolitis, cystic fibrosis, and emphysema that can cause constriction and/or obstruction of the airways.) </font>

We need to close those gaps so that we can segment the whole lung portion. With this view in mind, we can run closing operation so that it will fill up those portions. 

In [ ]:
from skimage.morphology import opening, closing, binary_dilation
from skimage.morphology import disk

def plot_comparison(original, filtered, filter_name):

    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4), sharex=True,
                                   sharey=True)
    ax1.imshow(original, cmap=plt.cm.gray)
    ax1.set_title('original')
    ax1.axis('off')
    ax2.imshow(filtered, cmap=plt.cm.gray)
    ax2.set_title(filter_name)
    ax2.axis('off')

In [ ]:
original = segmented_lungs_fill[46]

rows = 4
cols = 4
f, ax = plt.subplots(rows, cols, figsize = (15,12))

for i in range(rows*cols):
    if i==0:
        ax[0,0].imshow(original, cmap = plt.cm.gray)
        ax[0,0].set_title('Original')
        ax[0,0].axis(False)
    else:
        closed = closing(original, disk(i))
        ax[int(i/rows),int(i % rows)].set_title(f'closed disk({i})')
        ax[int(i/rows),int(i % rows)].imshow(closed, cmap = plt.cm.gray)
        ax[int(i/rows),int(i % rows)].axis('off')
plt.show()   

Therefore we can select the desired filters from here and just multiply that with the original image to get lung segmentation from these. Before selecting any particular filter, let's see the segmentation quality of the filters and then we can select the desired filter.

In [ ]:
original_image = patient_pixels[46]
original = segmented_lungs_fill[46]
f, ax = plt.subplots(rows, cols, figsize = (15,15))

for i in range(rows*cols):
    if i==0:
        ax[0,0].imshow(original_image, cmap = plt.cm.gray)
        ax[0,0].set_title('Original')
        ax[0,0].axis(False)
    else:
        closed = closing(original, disk(i))
        ax[int(i/rows),int(i % rows)].set_title(f'closed with disk({i})')
        ax[int(i/rows),int(i % rows)].imshow(original_image * closed, cmap = plt.cm.gray)
        ax[int(i/rows),int(i % rows)].axis('off')
plt.show()   

Now using the disk size 20 we again we perform closing operation on all the corresponding segmentation mask and produce output in the following  section of code.

In [ ]:
segmented_output = [image * binary_dilation(closing(mask, disk(20)), disk(4)) 
                    for image, mask in zip(patient_pixels, segmented_lungs_fill )]

In [ ]:
print('Segmented Part of Lung Tissue')
imageio.mimsave(f'segmented_output_{patient_id}.gif', segmented_output, duration=0.1)
display.Image(f'segmented_output_{patient_id}.gif', format='png')

In [ ]:
rows = 4
cols = 4
f, ax = plt.subplots(rows, cols, figsize = (15,15))

for i in range(rows*cols):
    ax[int(i/rows),int(i % rows)].set_title(f'slice({i*2+25})')
    ax[int(i/rows),int(i % rows)].imshow(segmented_output[i*2+25], cmap = plt.cm.gray)
    ax[int(i/rows),int(i % rows)].axis('off')
plt.show()

In [ ]:
unique_patient_profile